In [71]:
import os
import pickle
import numpy as np
from tqdm import tqdm
from collections import Counter
from torch.utils.data import DataLoader, TensorDataset

import torch
from transformers import RobertaTokenizer, RobertaModel
from transformers import AutoTokenizer, AutoModelForMaskedLM
from multiprocessing import Pool, cpu_count
from transformers import BertTokenizer, BertForSequenceClassification

os.environ['CUDA_VISIBLE_DEVICES'] = "1"

RELATION_INPUT_PATH = './data/3_openKE/synthesize/relation2id.txt'
RELATION_INPUT_LABEL_PATH = './data/3_openKE/synthesize/relation2id_type.txt'

RELATION_OUTPUT_PATH = './data/4_embedding/synthesize/secureBERT/relation.npy'
MODEL_SAVE_PATH = './data/4_embedding/synthesize/model/secureBERT/relation/'
if not os.path.exists(MODEL_SAVE_PATH):
    os.makedirs(MODEL_SAVE_PATH)

In [72]:
X = list()
Y = list()
with open(RELATION_INPUT_LABEL_PATH, 'r') as f:
    for row in f.readlines()[1:]:
        cat, rid = row.split('\t')
        Y.append(cat)
        
with open(RELATION_INPUT_PATH, 'r') as f:
    for row in f.readlines()[1:]:
        value, rid = row.split('\t')
        X.append(value)

print(f'Sentence Cnt: {len(X)}' , X[:5])
# print(f'Label Cnt: {len(Y)}', Y[:5])
print(f'Label Cnt: {len(Y)}', Y)

Sentence Cnt: 23 ['RegDeleteKey', 'Process Create', 'ReadFile', 'TCP Accept', 'RegOpenKey']
Label Cnt: 23 ['registry', 'process', 'file', 'network', 'registry', 'file', 'network', 'file', 'file', 'registry', 'file', 'registry', 'file', 'network', 'file', 'file', 'unknown', 'process', 'network', 'file', 'file', 'file', 'network']


In [73]:
# Convert category labels to tensor
# label_dict = {'file':0, 'registry':1, 'network':2, 'process':3}
label_dict = {'file':0, 'registry':1, 'network':2, 'process':3, 'unknown':4}

labels = torch.tensor([label_dict[y] for y in Y])

In [74]:
# SecureBERT
tokenizer = RobertaTokenizer.from_pretrained("ehsanaghaei/SecureBERT")
encoded_inputs = tokenizer.batch_encode_plus(
    X,
    padding='max_length',
    truncation=True,
    max_length=32,
    return_tensors='pt'
)

input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']

In [75]:
print(input_ids.shape)
print(attention_mask.shape)

torch.Size([23, 32])
torch.Size([23, 32])


In [76]:
# Move data to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

In [77]:
batch_size = 8

In [78]:
# splitter_tmp = int(0.9 * len(input_ids))
splitter1 = int(0.8 * len(input_ids))
train_inputs, val_inputs= input_ids, input_ids[splitter1:]
train_masks, val_masks= attention_mask, attention_mask[splitter1:]
train_labels, val_labels = labels, labels[splitter1:]
print(train_inputs.shape, Counter(Y))
print(val_inputs.shape, Counter(Y[splitter1:]))

# Create data loaders
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=False)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

torch.Size([23, 32]) Counter({'file': 11, 'network': 5, 'registry': 4, 'process': 2, 'unknown': 1})
torch.Size([5, 32]) Counter({'file': 3, 'network': 2})


In [79]:
# num_labels is important -> remember to modify
model = BertForSequenceClassification.from_pretrained("ehsanaghaei/SecureBERT", num_labels=5, output_hidden_states=True)
# Move the model to GPU
model.to('cuda')

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at ehsanaghaei/SecureBERT were not used when initializing BertForSequenceClassification: ['roberta.embeddings.token_type_embeddings.weight', 'roberta.encoder.layer.8.attention.self.query.weight', 'roberta.encoder.layer.7.attention.self.key.bias', 'lm_head.layer_norm.bias', 'roberta.encoder.layer.6.attention.self.key.bias', 'roberta.encoder.layer.10.attention.self.query.weight', 'roberta.encoder.layer.6.intermediate.dense.bias', 'roberta.encoder.layer.8.attention.self.value.weight', 'roberta.encoder.layer.6.attention.self.value.weight', 'roberta.encoder.layer.5.intermediate.dense.weight', 'roberta.encoder.layer.11.attention.self.key.weight', 'roberta.encoder.layer.1.attention.output.LayerNorm.weight', 'roberta.encoder.layer.2.attention.self.value.weight', 'lm_head.layer_norm.weight', 'roberta.encoder.

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ehsanaghaei/SecureBERT and are newly initialized: ['encoder.layer.6.attention.self.key.bias', 'encoder.layer.8.attention.output.LayerNorm.weight', 'encoder.layer.9.output.dense.bias', 'encoder.layer.2.attention.self.value.weight', 'encoder.layer.3.attention.output.dense.bias', 'encoder.layer.5.attention.self.value.bias', 'embeddings.LayerNorm.bias', 'encoder.layer.3.intermediate.dense.bias', 'encoder.layer.5.attention.self.key.weight', 'embeddings.LayerNorm.weight', 'encoder.layer.1.attention.output.dense.weight', 'encoder.layer.6.attention.self.value.weight', 'encoder.layer.10.attention.self.query.bias', 'encoder.layer.9.attention.self.key.weight', 'encoder.layer.3.attention.self.value.weight', 'encoder.layer.8.output.LayerNorm.bias', 'encoder.layer.11.attention.output.LayerNorm.weight', 'encoder.layer.3.intermediate.dense.weight', 'encoder.layer.7.output.dense.weight', 'encoder.layer.9.att

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05, element

In [81]:
# Fine-tune the BERT model on my task: 
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
epochs = 50
batch_chunk_size = 10000  # Number of batches to process before saving to disk

# Training loop
min_val_loss = 999999
for epoch in range(epochs):
    model.train()
    embeddings = []
#     print(f'Epoch {epoch+1} start!!!!!!!')
    for batch_idx, batch in enumerate(tqdm(train_dataloader)):
#         batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        
        # Extract the embeddings from the model's hidden states
        hidden_states = model.bert(input_ids, attention_mask=attention_mask)[2]
        last_hidden_layer_embeddings = hidden_states[-1].detach().cpu().numpy()
        embeddings.append(last_hidden_layer_embeddings)
    
    embeddings = np.concatenate(embeddings)
    
    # embeddings[sequence number, sequence length(26), hidden dimension(768)]
    embeddings = embeddings[:, 0, :]  # Take the embedding of the [CLS] token -> output = [26, 768]
    
    embeddings = np.array(embeddings)
    np.save(RELATION_OUTPUT_PATH, embeddings)
    print(f'{RELATION_OUTPUT_PATH}, shape={embeddings.shape}')
        
    with torch.no_grad():
        model.eval()
        
        val_loss = 0
        val_correct = 0
        val_total = 0

        for batch in val_dataloader:
            batch = tuple(t.to(device) for t in batch)
            input_ids, attention_mask, labels = batch

            with torch.no_grad():
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                logits = outputs.logits

            val_loss += loss.item()
            _, predicted_labels = torch.max(logits, dim=1)
            val_correct += (predicted_labels == labels).sum().item()
            val_total += labels.size(0)

    val_accuracy = val_correct / val_total
    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch: {epoch + 1}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
    
    if avg_val_loss < min_val_loss:
        # Save the trained model
        model.save_pretrained(MODEL_SAVE_PATH)
        min_val_loss = avg_val_loss
        
#     print(f'Epoch {epoch+1} end!!!!!!!')

100%|██████████| 3/3 [00:00<00:00, 12.08it/s]


./data/4_embedding/synthesize/secureBERT/relation.npy, shape=(23, 768)
Epoch: 1, Val Loss: 0.4458, Val Accuracy: 0.8000


100%|██████████| 3/3 [00:00<00:00, 11.54it/s]


./data/4_embedding/synthesize/secureBERT/relation.npy, shape=(23, 768)
Epoch: 2, Val Loss: 0.5651, Val Accuracy: 0.8000


100%|██████████| 3/3 [00:00<00:00, 12.30it/s]


...

./data/4_embedding/synthesize/secureBERT/relation.npy, shape=(23, 768)
Epoch: 48, Val Loss: 0.0038, Val Accuracy: 1.0000


100%|██████████| 3/3 [00:00<00:00, 10.19it/s]


./data/4_embedding/synthesize/secureBERT/relation.npy, shape=(23, 768)
Epoch: 49, Val Loss: 0.0037, Val Accuracy: 1.0000


100%|██████████| 3/3 [00:00<00:00, 10.61it/s]


./data/4_embedding/synthesize/secureBERT/relation.npy, shape=(23, 768)
Epoch: 50, Val Loss: 0.0037, Val Accuracy: 1.0000


#### The complete code

In [ ]:
import os
import pickle
import numpy as np
from tqdm import tqdm
from collections import Counter
from torch.utils.data import DataLoader, TensorDataset

import torch
from transformers import RobertaTokenizer, RobertaModel
from transformers import AutoTokenizer, AutoModelForMaskedLM
from multiprocessing import Pool, cpu_count
from transformers import BertTokenizer, BertForSequenceClassification

os.environ['CUDA_VISIBLE_DEVICES'] = "1"

RELATION_INPUT_PATH = './data/3_openKE/synthesize/relation2id.txt'
RELATION_INPUT_LABEL_PATH = './data/3_openKE/synthesize/relation2id_type.txt'
RELATION_OUTPUT_PATH = './data/4_embedding/synthesize/secureBERT/relation.npy'
MODEL_SAVE_PATH = './data/4_embedding/synthesize/model/secureBERT/relation/'
if not os.path.exists(MODEL_SAVE_PATH):
    os.makedirs(MODEL_SAVE_PATH)
    

X = list()
Y = list()
with open(RELATION_INPUT_LABEL_PATH, 'r') as f:
    for row in f.readlines()[1:]:
        cat, rid = row.split('\t')
        Y.append(cat)
        
with open(RELATION_INPUT_PATH, 'r') as f:
    for row in f.readlines()[1:]:
        value, rid = row.split('\t')
        X.append(value)

print(f'Sentence Cnt: {len(X)}' , X[:5])
# print(f'Label Cnt: {len(Y)}', Y[:5])
print(f'Label Cnt: {len(Y)}', Y)

label_dict = {'file':0, 'registry':1, 'network':2, 'process':3, 'unknown':4}


labels = torch.tensor([label_dict[y] for y in Y])

# SecureBERT
tokenizer = RobertaTokenizer.from_pretrained("ehsanaghaei/SecureBERT")
encoded_inputs = tokenizer.batch_encode_plus(
    X,
    padding='max_length',
    truncation=True,
    max_length=32,
    return_tensors='pt'
)

input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']



print(input_ids.shape)
print(attention_mask.shape)

# Move data to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

batch_size = 8

# splitter_tmp = int(0.9 * len(input_ids))
splitter1 = int(0.8 * len(input_ids))
train_inputs, val_inputs= input_ids, input_ids[splitter1:]
train_masks, val_masks= attention_mask, attention_mask[splitter1:]
train_labels, val_labels = labels, labels[splitter1:]
print(train_inputs.shape, Counter(Y))
print(val_inputs.shape, Counter(Y[splitter1:]))

# Create data loaders
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=False)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# num_labels is important -> remember to modify
model = BertForSequenceClassification.from_pretrained("ehsanaghaei/SecureBERT", num_labels=5, output_hidden_states=True)
# Move the model to GPU
model.to('cuda')


# Fine-tune the BERT model on my task: 
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
epochs = 50
batch_chunk_size = 10000  # Number of batches to process before saving to disk

# Training loop
min_val_loss = 999999
for epoch in range(epochs):
    model.train()
    embeddings = []
#     print(f'Epoch {epoch+1} start!!!!!!!')
    for batch_idx, batch in enumerate(tqdm(train_dataloader)):
#         batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        
        # Extract the embeddings from the model's hidden states
        hidden_states = model.bert(input_ids, attention_mask=attention_mask)[2]
        last_hidden_layer_embeddings = hidden_states[-1].detach().cpu().numpy()
        embeddings.append(last_hidden_layer_embeddings)
    
    embeddings = np.concatenate(embeddings)
    
    # embeddings[sequence number, sequence length(26), hidden dimension(768)]
    embeddings = embeddings[:, 0, :]  # Take the embedding of the [CLS] token -> output = [26, 768]
    
    embeddings = np.array(embeddings)
    np.save(RELATION_OUTPUT_PATH, embeddings)
    print(f'{RELATION_OUTPUT_PATH}, shape={embeddings.shape}')
        
    with torch.no_grad():
        model.eval()
        
        val_loss = 0
        val_correct = 0
        val_total = 0

        for batch in val_dataloader:
            batch = tuple(t.to(device) for t in batch)
            input_ids, attention_mask, labels = batch

            with torch.no_grad():
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                logits = outputs.logits

            val_loss += loss.item()
            _, predicted_labels = torch.max(logits, dim=1)
            val_correct += (predicted_labels == labels).sum().item()
            val_total += labels.size(0)

    val_accuracy = val_correct / val_total
    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch: {epoch + 1}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
    
    if avg_val_loss < min_val_loss:
        # Save the trained model
        model.save_pretrained(MODEL_SAVE_PATH)
        min_val_loss = avg_val_loss
        
#     print(f'Epoch {epoch+1} end!!!!!!!')

#### the complete code

In [12]:
import os
import pickle
import numpy as np
from tqdm import tqdm
from collections import Counter
from torch.utils.data import DataLoader, TensorDataset

import torch
from transformers import RobertaTokenizer, RobertaModel
from transformers import AutoTokenizer, AutoModelForMaskedLM
from multiprocessing import Pool, cpu_count
from transformers import BertTokenizer, BertForSequenceClassification

os.environ['CUDA_VISIBLE_DEVICES'] = "3"

RELATION_INPUT_PATH = './data/3_openKE/synthesize/relation2id.txt'
RELATION_INPUT_LABEL_PATH = './data/3_openKE/synthesize/relation2id_type.txt'
RELATION_OUTPUT_PATH = './data/4_embedding/synthesize/secureBERT/relation.npy'
MODEL_SAVE_PATH = './data/4_embedding/synthesize/model/secureBERT/relation/'
if not os.path.exists(MODEL_SAVE_PATH):
    os.makedirs(MODEL_SAVE_PATH)

    
X = list()
Y = list()
with open(RELATION_INPUT_LABEL_PATH, 'r') as f:
    for row in f.readlines()[1:]:
        cat, rid = row.split('\t')
        Y.append(cat)
with open(RELATION_INPUT_PATH, 'r') as f:
    for row in f.readlines()[1:]:
        value, rid = row.split('\t')
        X.append(value)

print(f'Sentence Cnt: {len(X)}' , X[:5])
print(f'Label Cnt: {len(Y)}', Y[:5])


# Convert category labels to tensor
label_dict = {'file':0, 'registry':1, 'network':2, 'process':3}
labels = torch.tensor([label_dict[y] for y in Y])


# SecureBERT
tokenizer = RobertaTokenizer.from_pretrained("ehsanaghaei/SecureBERT")
encoded_inputs = tokenizer.batch_encode_plus(
    X,
    padding='max_length',
    truncation=True,
    max_length=32,
    return_tensors='pt'
)

input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']


print(input_ids.shape)
print(attention_mask.shape)


# Move data to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)


batch_size = 8


# splitter_tmp = int(0.9 * len(input_ids))
splitter1 = int(0.8 * len(input_ids))
train_inputs, val_inputs= input_ids, input_ids[splitter1:]
train_masks, val_masks= attention_mask, attention_mask[splitter1:]
train_labels, val_labels = labels, labels[splitter1:]
print(train_inputs.shape, Counter(Y))
print(val_inputs.shape, Counter(Y[splitter1:]))

# Create data loaders
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=False)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)


model = BertForSequenceClassification.from_pretrained("ehsanaghaei/SecureBERT", num_labels=4, output_hidden_states=True)
# Move the model to GPU
model.to('cuda')

# Fine-tune the BERT model on my task:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
epochs = 10
batch_chunk_size = 10000  # Number of batches to process before saving to disk

# Training loop
min_val_loss = 999999
for epoch in range(epochs):
    model.train()
    embeddings = []
#     print(f'Epoch {epoch+1} start!!!!!!!')
    for batch_idx, batch in enumerate(tqdm(train_dataloader)):
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        
        # Extract the embeddings from the model's hidden states
        hidden_states = model.bert(input_ids, attention_mask=attention_mask)[2]
        last_hidden_layer_embeddings = hidden_states[-1].detach().cpu().numpy()
        embeddings.append(last_hidden_layer_embeddings)
    
    embeddings = np.concatenate(embeddings)
    embeddings = embeddings[:, 0, :]  # Take the embedding of the [CLS] token
    embeddings = np.array(embeddings)
    np.save(RELATION_OUTPUT_PATH, embeddings)
    print(f'{RELATION_OUTPUT_PATH}, shape={embeddings.shape}')
        
    with torch.no_grad():
        model.eval()
        
        val_loss = 0
        val_correct = 0
        val_total = 0

        for batch in val_dataloader:
            batch = tuple(t.to(device) for t in batch)
            input_ids, attention_mask, labels = batch

            with torch.no_grad():
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                logits = outputs.logits

            val_loss += loss.item()
            _, predicted_labels = torch.max(logits, dim=1)
            val_correct += (predicted_labels == labels).sum().item()
            val_total += labels.size(0)

    val_accuracy = val_correct / val_total
    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch: {epoch + 1}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
    
    if avg_val_loss < min_val_loss:
        # Save the trained model
        model.save_pretrained(MODEL_SAVE_PATH)
        min_val_loss = avg_val_loss
        
#     print(f'Epoch {epoch+1} end!!!!!!!')





Sentence Cnt: 26 ['QueryAllInformationFile', 'QueryNetworkOpenInformationFile', 'RegSetValue', 'CreateFile', 'SetDispositionInformationFile']
Label Cnt: 26 ['file', 'network', 'registry', 'file', 'file']
torch.Size([26, 32])
torch.Size([26, 32])
torch.Size([26, 32]) Counter({'file': 11, 'network': 7, 'registry': 7, 'process': 1})
torch.Size([6, 32]) Counter({'file': 3, 'network': 2, 'process': 1})


You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at ehsanaghaei/SecureBERT were not used when initializing BertForSequenceClassification: ['roberta.encoder.layer.11.attention.output.LayerNorm.bias', 'roberta.encoder.layer.2.attention.output.LayerNorm.weight', 'roberta.encoder.layer.5.output.LayerNorm.bias', 'roberta.encoder.layer.11.attention.output.LayerNorm.weight', 'roberta.encoder.layer.5.attention.output.LayerNorm.bias', 'roberta.encoder.layer.5.output.dense.bias', 'roberta.encoder.layer.9.attention.self.key.bias', 'roberta.encoder.layer.9.attention.self.value.weight', 'roberta.encoder.layer.3.attention.self.key.bias', 'roberta.encoder.layer.3.attention.output.dense.weight', 'roberta.encoder.layer.8.attention.output.dense.weight', 'roberta.encoder.layer.3.output.dense.bias', 'roberta.encoder.layer.3.attention.output.LayerNorm.bias', 'roberta.e

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ehsanaghaei/SecureBERT and are newly initialized: ['encoder.layer.9.output.dense.weight', 'encoder.layer.9.attention.self.query.weight', 'encoder.layer.2.attention.output.LayerNorm.bias', 'encoder.layer.2.attention.self.value.bias', 'pooler.dense.bias', 'encoder.layer.4.attention.self.query.bias', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.5.attention.output.dense.bias', 'encoder.layer.9.attention.self.value.weight', 'encoder.layer.9.intermediate.dense.bias', 'encoder.layer.5.attention.self.value.bias', 'encoder.layer.4.attention.self.value.weight', 'encoder.layer.10.intermediate.dense.weight', 'encoder.layer.9.output.LayerNorm.weight', 'encoder.layer.5.attention.self.value.weight', 'encoder.layer.2.attention.output.dense.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.7.attention.output.dense.bias', 'encoder.layer.11.output.dense.bias', 'encoder.layer.9.a

100%|██████████| 4/4 [00:00<00:00, 11.58it/s]


./data/4_embedding/synthesize/secureBERT/relation.npy, shape=(26, 768)
Epoch: 1, Val Loss: 1.2403, Val Accuracy: 0.8333


...

./data/4_embedding/synthesize/secureBERT/relation.npy, shape=(26, 768)
Epoch: 8, Val Loss: 0.0854, Val Accuracy: 1.0000


100%|██████████| 4/4 [00:00<00:00, 11.99it/s]


./data/4_embedding/synthesize/secureBERT/relation.npy, shape=(26, 768)
Epoch: 9, Val Loss: 0.0480, Val Accuracy: 1.0000


100%|██████████| 4/4 [00:00<00:00, 12.37it/s]


./data/4_embedding/synthesize/secureBERT/relation.npy, shape=(26, 768)
Epoch: 10, Val Loss: 0.0362, Val Accuracy: 1.0000
